## Language Translation Using sequence to sequence 

In [160]:
#importing libraries
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

In [161]:
# Path to the data txt file on disk.
data_path = r'D:\data science\Projects that cannot be comitted\Language translation using Sequence to sequence from english to french\fra.txt'
number_samples = 10000       # Number of samples to train on.

In [162]:

latent_dimension = 256  # Latent dimensionality of the encoding space.


In [163]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(number_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [164]:
input_texts

['Go.',
 'Hi.',
 'Hi.',
 'Run!',
 'Run!',
 'Who?',
 'Wow!',
 'Fire!',
 'Help!',
 'Jump.',
 'Stop!',
 'Stop!',
 'Stop!',
 'Wait!',
 'Wait!',
 'Go on.',
 'Go on.',
 'Go on.',
 'Hello!',
 'Hello!',
 'I see.',
 'I try.',
 'I won!',
 'I won!',
 'I won.',
 'Oh no!',
 'Attack!',
 'Attack!',
 'Cheers!',
 'Cheers!',
 'Cheers!',
 'Cheers!',
 'Get up.',
 'Go now.',
 'Go now.',
 'Go now.',
 'Got it!',
 'Got it!',
 'Got it?',
 'Got it?',
 'Got it?',
 'Hop in.',
 'Hop in.',
 'Hug me.',
 'Hug me.',
 'I fell.',
 'I fell.',
 'I know.',
 'I left.',
 'I left.',
 'I lied.',
 'I lost.',
 'I paid.',
 "I'm 19.",
 "I'm OK.",
 "I'm OK.",
 'Listen.',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'Really?',
 'Really?',
 'Really?',
 'Thanks.',
 'We try.',
 'We won.',
 'We won.',
 'We won.',
 'We won.',
 'Ask Tom.',
 'Awesome!',
 'Be calm.',
 'Be calm.',
 'Be calm.',
 'Be cool.',
 'Be fair.',
 'Be fair.',
 'Be fair.',
 'Be fair.',
 'Be fair.',
 'Be fai

In [165]:
target_texts

['\tVa !\n',
 '\tSalut !\n',
 '\tSalut.\n',
 '\tCours\u202f!\n',
 '\tCourez\u202f!\n',
 '\tQui ?\n',
 '\tÇa alors\u202f!\n',
 '\tAu feu !\n',
 "\tÀ l'aide\u202f!\n",
 '\tSaute.\n',
 '\tÇa suffit\u202f!\n',
 '\tStop\u202f!\n',
 '\tArrête-toi !\n',
 '\tAttends !\n',
 '\tAttendez !\n',
 '\tPoursuis.\n',
 '\tContinuez.\n',
 '\tPoursuivez.\n',
 '\tBonjour !\n',
 '\tSalut !\n',
 '\tJe comprends.\n',
 "\tJ'essaye.\n",
 "\tJ'ai gagné !\n",
 "\tJe l'ai emporté !\n",
 '\tJ’ai gagné.\n',
 '\tOh non !\n',
 '\tAttaque !\n',
 '\tAttaquez !\n',
 '\tSanté !\n',
 '\tÀ votre santé !\n',
 '\tMerci !\n',
 '\tTchin-tchin !\n',
 '\tLève-toi.\n',
 '\tVa, maintenant.\n',
 '\tAllez-y maintenant.\n',
 '\tVas-y maintenant.\n',
 "\tJ'ai pigé !\n",
 '\tCompris !\n',
 '\tPigé\u202f?\n',
 '\tCompris\u202f?\n',
 "\tT'as capté\u202f?\n",
 '\tMonte.\n',
 '\tMontez.\n',
 '\tSerre-moi dans tes bras !\n',
 '\tSerrez-moi dans vos bras !\n',
 '\tJe suis tombée.\n',
 '\tJe suis tombé.\n',
 '\tJe sais.\n',
 '\tJe suis parti.\

In [166]:
input_characters


{' ',
 '!',
 '$',
 '%',
 '&',
 "'",
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [167]:
len(input_characters)

70

In [168]:
target_characters


{'\t',
 '\n',
 ' ',
 '!',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '5',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'x',
 'y',
 'z',
 '\xa0',
 '«',
 '»',
 'À',
 'Ç',
 'É',
 'Ê',
 'à',
 'â',
 'ç',
 'è',
 'é',
 'ê',
 'ë',
 'î',
 'ï',
 'ô',
 'ù',
 'û',
 'œ',
 '\u2009',
 '’',
 '\u202f'}

In [169]:
len(target_characters)

93

# sorting input character and target characters 

In [170]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

In [171]:
input_characters

[' ',
 '!',
 '$',
 '%',
 '&',
 "'",
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [172]:
target_characters

['\t',
 '\n',
 ' ',
 '!',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '5',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'x',
 'y',
 'z',
 '\xa0',
 '«',
 '»',
 'À',
 'Ç',
 'É',
 'Ê',
 'à',
 'â',
 'ç',
 'è',
 'é',
 'ê',
 'ë',
 'î',
 'ï',
 'ô',
 'ù',
 'û',
 'œ',
 '\u2009',
 '’',
 '\u202f']

In [173]:
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of unique input tokens: 70
Number of unique output tokens: 93
Max sequence length for inputs: 16
Max sequence length for outputs: 59


In [174]:
# here whatever unique token we have, unique number is associated with this same for input and target tokens
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)]
)

target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)]
)


In [175]:
input_token_index

{' ': 0,
 '!': 1,
 '$': 2,
 '%': 3,
 '&': 4,
 "'": 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '0': 9,
 '1': 10,
 '2': 11,
 '3': 12,
 '5': 13,
 '6': 14,
 '7': 15,
 '8': 16,
 '9': 17,
 ':': 18,
 '?': 19,
 'A': 20,
 'B': 21,
 'C': 22,
 'D': 23,
 'E': 24,
 'F': 25,
 'G': 26,
 'H': 27,
 'I': 28,
 'J': 29,
 'K': 30,
 'L': 31,
 'M': 32,
 'N': 33,
 'O': 34,
 'P': 35,
 'Q': 36,
 'R': 37,
 'S': 38,
 'T': 39,
 'U': 40,
 'V': 41,
 'W': 42,
 'Y': 43,
 'a': 44,
 'b': 45,
 'c': 46,
 'd': 47,
 'e': 48,
 'f': 49,
 'g': 50,
 'h': 51,
 'i': 52,
 'j': 53,
 'k': 54,
 'l': 55,
 'm': 56,
 'n': 57,
 'o': 58,
 'p': 59,
 'q': 60,
 'r': 61,
 's': 62,
 't': 63,
 'u': 64,
 'v': 65,
 'w': 66,
 'x': 67,
 'y': 68,
 'z': 69}

In [176]:
target_token_index

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '$': 4,
 '%': 5,
 '&': 6,
 "'": 7,
 '(': 8,
 ')': 9,
 ',': 10,
 '-': 11,
 '.': 12,
 '0': 13,
 '1': 14,
 '2': 15,
 '3': 16,
 '5': 17,
 '8': 18,
 '9': 19,
 ':': 20,
 '?': 21,
 'A': 22,
 'B': 23,
 'C': 24,
 'D': 25,
 'E': 26,
 'F': 27,
 'G': 28,
 'H': 29,
 'I': 30,
 'J': 31,
 'K': 32,
 'L': 33,
 'M': 34,
 'N': 35,
 'O': 36,
 'P': 37,
 'Q': 38,
 'R': 39,
 'S': 40,
 'T': 41,
 'U': 42,
 'V': 43,
 'Y': 44,
 'a': 45,
 'b': 46,
 'c': 47,
 'd': 48,
 'e': 49,
 'f': 50,
 'g': 51,
 'h': 52,
 'i': 53,
 'j': 54,
 'k': 55,
 'l': 56,
 'm': 57,
 'n': 58,
 'o': 59,
 'p': 60,
 'q': 61,
 'r': 62,
 's': 63,
 't': 64,
 'u': 65,
 'v': 66,
 'x': 67,
 'y': 68,
 'z': 69,
 '\xa0': 70,
 '«': 71,
 '»': 72,
 'À': 73,
 'Ç': 74,
 'É': 75,
 'Ê': 76,
 'à': 77,
 'â': 78,
 'ç': 79,
 'è': 80,
 'é': 81,
 'ê': 82,
 'ë': 83,
 'î': 84,
 'ï': 85,
 'ô': 86,
 'ù': 87,
 'û': 88,
 'œ': 89,
 '\u2009': 90,
 '’': 91,
 '\u202f': 92}

# Creating encoder and decoder

In [177]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype = 'float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype = 'float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype = 'float32')



# Print the shape of the padded sequences to verify
print("Encoder input shape:", encoder_input_data.shape)
print("Decoder input shape:", decoder_input_data.shape)
print("Decoder target shape:", decoder_target_data.shape)

Encoder input shape: (10000, 16, 70)
Decoder input shape: (10000, 59, 93)
Decoder target shape: (10000, 59, 93)


In [178]:
# One hot encoding 
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [179]:
decoder_input_data

array([[[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]],

       [[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]],

       [[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]],

       ...,

       [[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0.

In [180]:
decoder_target_data

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0.

In [181]:
# Padding sequences
from keras.preprocessing.sequence import pad_sequences
encoder_input_data = pad_sequences(encoder_input_data, maxlen=max_encoder_seq_length, dtype='float32', padding='post')
decoder_input_data = pad_sequences(decoder_input_data, maxlen=max_decoder_seq_length, dtype='float32', padding='post')
decoder_target_data = pad_sequences(decoder_target_data, maxlen=max_decoder_seq_length, dtype='float32', padding='post')

## Encoder 

In [182]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dimension, return_state= True)  # latent dimentions is time stamps
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# we discard 'encoder_outputs' and only keep the states.
encoder_states = [state_h, state_c]

# Decoder 

In [183]:
# set up the decoder, using 'encoder_states' as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences, and to return internal states as well. We don't use the return
#states in the training model, but we will use them in inferences


# here return_sequences = True and return_states = True because it will take output of previous LSTM
decoder_lstm = LSTM(latent_dimension, return_sequences = True,  return_state= True)  


decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state= encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation = 'softmax' )
decoder_outputs = decoder_dense(decoder_outputs)


In [184]:
# Seq2seq model 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [185]:
# Run training

batch_s = 64  # Batch size for training.
epochs = 5  # Number of epochs to train for.
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_s,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/5


125/125 [==============================] - 43s 277ms/step - loss: 1.2648 - accuracy: 0.7200 - val_loss: 1.1652 - val_accuracy: 0.6962
Epoch 2/5
125/125 [==============================] - 34s 271ms/step - loss: 0.9758 - accuracy: 0.7361 - val_loss: 1.0471 - val_accuracy: 0.7073
Epoch 3/5
125/125 [==============================] - 34s 269ms/step - loss: 0.8818 - accuracy: 0.7573 - val_loss: 0.9303 - val_accuracy: 0.7394
Epoch 4/5
125/125 [==============================] - 34s 269ms/step - loss: 0.7761 - accuracy: 0.7829 - val_loss: 0.8288 - val_accuracy: 0.7642
Epoch 5/5
125/125 [==============================] - 33s 268ms/step - loss: 0.6994 - accuracy: 0.7989 - val_loss: 0.7796 - val_accuracy: 0.7736


In [186]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dimension,))
decoder_state_input_c = Input(shape=(latent_dimension,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [187]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [188]:
reverse_input_char_index

{0: ' ',
 1: '!',
 2: '$',
 3: '%',
 4: '&',
 5: "'",
 6: ',',
 7: '-',
 8: '.',
 9: '0',
 10: '1',
 11: '2',
 12: '3',
 13: '5',
 14: '6',
 15: '7',
 16: '8',
 17: '9',
 18: ':',
 19: '?',
 20: 'A',
 21: 'B',
 22: 'C',
 23: 'D',
 24: 'E',
 25: 'F',
 26: 'G',
 27: 'H',
 28: 'I',
 29: 'J',
 30: 'K',
 31: 'L',
 32: 'M',
 33: 'N',
 34: 'O',
 35: 'P',
 36: 'Q',
 37: 'R',
 38: 'S',
 39: 'T',
 40: 'U',
 41: 'V',
 42: 'W',
 43: 'Y',
 44: 'a',
 45: 'b',
 46: 'c',
 47: 'd',
 48: 'e',
 49: 'f',
 50: 'g',
 51: 'h',
 52: 'i',
 53: 'j',
 54: 'k',
 55: 'l',
 56: 'm',
 57: 'n',
 58: 'o',
 59: 'p',
 60: 'q',
 61: 'r',
 62: 's',
 63: 't',
 64: 'u',
 65: 'v',
 66: 'w',
 67: 'x',
 68: 'y',
 69: 'z'}

In [189]:
reverse_target_char_index

{0: '\t',
 1: '\n',
 2: ' ',
 3: '!',
 4: '$',
 5: '%',
 6: '&',
 7: "'",
 8: '(',
 9: ')',
 10: ',',
 11: '-',
 12: '.',
 13: '0',
 14: '1',
 15: '2',
 16: '3',
 17: '5',
 18: '8',
 19: '9',
 20: ':',
 21: '?',
 22: 'A',
 23: 'B',
 24: 'C',
 25: 'D',
 26: 'E',
 27: 'F',
 28: 'G',
 29: 'H',
 30: 'I',
 31: 'J',
 32: 'K',
 33: 'L',
 34: 'M',
 35: 'N',
 36: 'O',
 37: 'P',
 38: 'Q',
 39: 'R',
 40: 'S',
 41: 'T',
 42: 'U',
 43: 'V',
 44: 'Y',
 45: 'a',
 46: 'b',
 47: 'c',
 48: 'd',
 49: 'e',
 50: 'f',
 51: 'g',
 52: 'h',
 53: 'i',
 54: 'j',
 55: 'k',
 56: 'l',
 57: 'm',
 58: 'n',
 59: 'o',
 60: 'p',
 61: 'q',
 62: 'r',
 63: 's',
 64: 't',
 65: 'u',
 66: 'v',
 67: 'x',
 68: 'y',
 69: 'z',
 70: '\xa0',
 71: '«',
 72: '»',
 73: 'À',
 74: 'Ç',
 75: 'É',
 76: 'Ê',
 77: 'à',
 78: 'â',
 79: 'ç',
 80: 'è',
 81: 'é',
 82: 'ê',
 83: 'ë',
 84: 'î',
 85: 'ï',
 86: 'ô',
 87: 'ù',
 88: 'û',
 89: 'œ',
 90: '\u2009',
 91: '’',
 92: '\u202f'}

In [190]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [191]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

1/1 [==============================] - 0s 50ms/step
-
Input sentence: Go.
Decoded sentence: Il s s me s me s me s  me                                   
1/1 [==============================] - 0s 43ms/step
-
Input sentence: Hi.
Decoded sentence: Je suis pais pais !

1/1 [==============================] - 0s 47ms/step
-
Input sentence: Hi.
Decoded sentence: Je suis pais pais !

1/1 [==============================] - 0s 45ms/step
-
Input sentence: Run!
Decoded sentence: Tous sous  ai  ai le !

1/1 [==============================] - 0s 42ms/step
-
Input sentence: Run!
Decoded sentence: Tous sous  ai  ai le !

1/1 [==============================] - 0s 45ms/step
-
Input sentence: Who?
Decoded sentence: Il s s me s me s me s  me                                   
1/1 [==============================] - 0s 57ms/step
-
Input sentence: Wow!
Decoded sentence: Tous sous  ai  ai le !

1/1 [==============================] - 0s 45ms/step
-
Input sentence: Fire!
Decoded sentence: Tous sous  a          